In [1]:
from C_split_functions import *
from C_econ import *


def objective_function2(x, equipment):
    ntur = 85  # 2019 Nabil 93
    ncomp = 82  #  89
    fg_tin = 539.76  # °C 630
    fg_m = 68.75  # kg/s 935
    cooler_pdrop = 1e5  # 0.5e5
    heater_pdrop = 0  # 1e5
    hx_pdrop = 0.5e5  # 1e5
    PENALTY_VALUE = float(1e6)
    splitter = False
    splitter2 = False
    looping = False
    enumerated_equipment = list(enumerate(equipment))
    equipment_length = len(equipment)
    equipment_array = np.array(equipment)
    enthalpies = np.zeros(equipment_length)
    entropies = np.zeros(equipment_length)
    exergies = np.zeros(equipment_length)
    w_comp = np.zeros(equipment_length)
    cost_comp = np.zeros(equipment_length)
    w_tur = np.zeros(equipment_length)
    cost_tur = np.zeros(equipment_length)
    q_cooler = np.zeros(equipment_length)
    cost_cooler = np.zeros(equipment_length)
    dissipation = np.zeros(equipment_length)
    q_heater = np.zeros(equipment_length)
    cost_heater = np.zeros(equipment_length)
    q_hx = np.zeros(equipment_length)
    cost_hx = np.zeros(equipment_length)
    cost_hx2= np.zeros(equipment_length)
    if 9 in equipment:
        splitter = True
        if 6 in equipment:
            looping = True
    if 10 in equipment:
        splitter2 = True
        looping = True
    
    def inlet_equipment_check(
        equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
    ):
        if equipment[assumed_inlet] == 7:
            splitter_position = np.where(equipment_array == 9)[0]
            mixer1, mixer2 = np.where(equipment_array == 7)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 8:
            splitter_position = np.where(equipment_array == 10)[0]
            mixer1, mixer2 = np.where(equipment_array == 8)[0]
            if mixer1 == assumed_inlet:
                Temperatures[splitter_position] = Temperatures[mixer1]
                enthalpies[splitter_position] = enthalpies[mixer1]
                entropies[splitter_position] = entropies[mixer1]

        if equipment[assumed_inlet] == 9:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]

        if equipment[assumed_inlet] == 10:
            splitter_position = np.where(equipment_array == 9)[0]
            Temperatures[splitter_position - 1] = Temperatures[assumed_inlet]
            enthalpies[splitter_position - 1] = enthalpies[assumed_inlet]
            entropies[splitter_position - 1] = entropies[assumed_inlet]
        return Temperatures, enthalpies, entropies

    def c_solver_splitter_mixer_calc(splitter_position,mixer1,mixer2,Temperatures,Pressures,enthalpies,entropies,mass_flow):
        if Temperatures[splitter_position - 1] != 0: 
            Temperatures[splitter_position] = Temperatures[splitter_position - 1]
            enthalpies[splitter_position] = enthalpies[splitter_position - 1]
            entropies[splitter_position] = entropies[splitter_position - 1]
        # if Temperatures[mixer1] != 0:
        #     Temperatures[splitter_position] = Temperatures[mixer1]
        #     enthalpies[splitter_position] = enthalpies[mixer1]
        #     entropies[splitter_position] = entropies[mixer1]
        Temperatures[mixer1] = Temperatures[splitter_position]
        enthalpies[mixer1] = enthalpies[splitter_position]
        entropies[mixer1] = entropies[splitter_position]
        if (
            Pressures[mixer1 - 1] == Pressures[mixer2 - 1]
            and Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            inlet1 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer1 - 1]),
                Input.pressure(Pressures[mixer1 - 1]),
            )
            inlet2 = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[mixer2 - 1]),
                Input.pressure(Pressures[mixer2 - 1]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[mixer1 - 1], inlet1, mass_flow[mixer2 - 1], inlet2
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        if (
            Temperatures[mixer1 - 1] != 0
            and Temperatures[mixer2 - 1] != 0
        ):
            if Pressures[mixer1 - 1] > Pressures[mixer2 - 1]:
                hp_index = mixer1 - 1
                lp_index = mixer2 - 1
            elif Pressures[mixer1 - 1] < Pressures[mixer2 - 1]:
                hp_index = mixer2 - 1
                lp_index = mixer1 - 1
            hp_inlet = (
                Fluid(FluidsList.CarbonDioxide)
                .with_state(
                    Input.temperature(Temperatures[hp_index]),
                    Input.pressure(Pressures[hp_index]),
                )
                .isenthalpic_expansion_to_pressure(Pressures[lp_index])
            )
            lp_inlet = Fluid(FluidsList.CarbonDioxide).with_state(
                Input.temperature(Temperatures[lp_index]),
                Input.pressure(Pressures[lp_index]),
            )
            outlet = Fluid(FluidsList.CarbonDioxide).mixing(
                mass_flow[hp_index], hp_inlet, mass_flow[lp_index], lp_inlet
            )
            Temperatures[mixer2] = outlet.temperature
            enthalpies[mixer2] = outlet.enthalpy
            entropies[mixer2] = outlet.entropy
        return Temperatures, Pressures,enthalpies, entropies 
    
    (
        Pressures,
        Temperatures,
        approach_temp,
        approach_temp_2,
        split_ratio,
        split_ratio_2,
        mass_flow,
        assumed_temperature,
    ) = decision_variable_placement(x, enumerated_equipment, equipment_length,looping)
    if looping == True:
        assumed_temperature = x[-2]
    print(assumed_temperature,"initial assumption")
    Pressures = Pressure_calculation(
            Pressures, equipment, cooler_pdrop, heater_pdrop, hx_pdrop, splitter,splitter2
        )
    tur_pratio, comp_pratio = tur_comp_pratio(
        enumerated_equipment, Pressures, equipment_length
    )
    turbine_position = np.where(equipment_array == 1)[0]
    cooler_position = np.where(equipment_array==2)[0]
    for index in cooler_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    compressor_position = np.where(equipment_array == 3)[0]
    heater_position = np.where(equipment_array==4)[0]
    for index in heater_position:
        enthalpies[index], entropies[index] = enthalpy_entropy(
            Temperatures[index], Pressures[index]
        )
    hx_position = np.where(equipment_array==5)[0]
    hx2_position =np.where(equipment_array==6)[0]
    while_counter = 0
    while Temperatures.prod() == 0:
        # restructuring this part can be useful, separating splitter information from tur/comp calculation while adding if checks
        # combinnig two power checks within the if check
        (
            Temperatures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
        ) = turbine_compressor_calculation(
            Temperatures,
            Pressures,
            enthalpies,
            entropies,
            w_tur,
            w_comp,
            equipment,
            ntur,
            ncomp,
            mass_flow,
        )

        if splitter == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )
        if splitter2 == True:
            (Temperatures, enthalpies, entropies) = splitter_mixer_calc_2(
                Temperatures, Pressures, enthalpies, entropies, mass_flow, equipment
            )


        if (
            hx_position.size != 0
            and Temperatures[hx_position[0] - 1] != 0
            and Temperatures[hx_position[1] - 1] != 0
        ):
            hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
            if (
                Temperatures[hotside_index - 1]
                < Temperatures[coldside_index - 1] + approach_temp
            ):
                print("Infeasible HX1.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
            except:
                print("HX1 calculation error")
                return PENALTY_VALUE

        if (
            hx2_position.size != 0
            and Temperatures[hx2_position[0] - 1] != 0
            and Temperatures[hx2_position[1] - 1] != 0
        ):
            hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
            if (
                Temperatures[hotside_index2 - 1]
                < Temperatures[coldside_index2 - 1] + approach_temp_2
            ):
                print("Infeasible HX2.1")
                return PENALTY_VALUE
            try:
                (
                    Temperatures[hotside_index2],
                    enthalpies[hotside_index2],
                    entropies[hotside_index2],
                    Temperatures[coldside_index2],
                    enthalpies[coldside_index2],
                    entropies[coldside_index2],
                    q_hx[min(hotside_index2, coldside_index2)],
                ) = HX_calculation(
                    Temperatures[hotside_index2 - 1],
                    Pressures[hotside_index2 - 1],
                    enthalpies[hotside_index2 - 1],
                    Temperatures[coldside_index2 - 1],
                    Pressures[coldside_index2 - 1],
                    enthalpies[coldside_index2 - 1],
                    approach_temp_2,
                    hx_pdrop,
                    mass_flow[hotside_index2],
                    mass_flow[coldside_index2],
                )
            except:
                print("HX2 calculation error")
                return PENALTY_VALUE
        if while_counter == 3:
            # print("Infeasible Temperatures")
            break
        while_counter += 1
    if splitter == True:
        splitter1_position = np.where(equipment_array == 9)[0]
        mixer1, mixer2 = np.where(equipment_array == 7)[0]
    if splitter2 == True:
        splitter2_position = np.where(equipment_array == 10)[0]
        mixer3, mixer4 = np.where(equipment_array == 8)[0]
    print(Temperatures,"Initial Temperatures")
    if Temperatures.prod() == 0 and looping == True and Temperatures[hx_position-1].sum() > 0:
        print("Entering Looping")
        assumed_inlet = hx_position[np.where(Temperatures[hx_position-1]==0)[0].item()]-1
        Temperatures[assumed_inlet] = assumed_temperature
        enthalpies[assumed_inlet], entropies[assumed_inlet] = enthalpy_entropy(Temperatures[assumed_inlet], Pressures[assumed_inlet])
        #Branch check
        Temperatures, enthalpies, entropies = inlet_equipment_check(
            equipment_array, assumed_inlet, Temperatures, enthalpies, entropies
        )
        hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
        if (
            Temperatures[hotside_index - 1]
            > Temperatures[coldside_index - 1] + approach_temp
        ):
            try:
                (
                    Temperatures[hotside_index],
                    enthalpies[hotside_index],
                    entropies[hotside_index],
                    Temperatures[coldside_index],
                    enthalpies[coldside_index],
                    entropies[coldside_index],
                    q_hx[min(hotside_index, coldside_index)],
                ) = HX_calculation(
                    Temperatures[hotside_index - 1],
                    Pressures[hotside_index - 1],
                    enthalpies[hotside_index - 1],
                    Temperatures[coldside_index - 1],
                    Pressures[coldside_index - 1],
                    enthalpies[coldside_index - 1],
                    approach_temp,
                    hx_pdrop,
                    mass_flow[hotside_index],
                    mass_flow[coldside_index],
                )
                hx_converged = True
                # print(Temperatures,"HX1 Converged Temperatures")
            except:
                return PENALTY_VALUE
            if hx_converged == True:
                for while_counter2nd in range(5):
                    # print(while_counter2nd,Temperatures)
                    for index in turbine_position:
                        if Temperatures[index-1] != 0:
                            (
                                enthalpies[index],
                                entropies[index],
                                Temperatures[index],
                                w_tur[index],
                            ) = turbine(
                                Temperatures[index-1],
                                Pressures[index-1],
                                Pressures[index],
                                ntur,
                                mass_flow[index],
                            )
                    for index in compressor_position:
                        if Temperatures[index-1] != 0:
                            (
                                enthalpies[index],
                                entropies[index],
                                Temperatures[index],
                                w_comp[index],
                            ) = compressor(
                                Temperatures[index-1],
                                Pressures[index-1],
                                Pressures[index],
                                ncomp,
                                mass_flow[index],
                            )
                    if splitter == True:
                        Temperatures,Pressures,enthalpies,entropies = c_solver_splitter_mixer_calc(splitter1_position,mixer1,mixer2,Temperatures,Pressures,enthalpies,entropies,mass_flow)
                    if splitter2 == True:
                        Temperatures,Pressures,enthalpies,entropies = c_solver_splitter_mixer_calc(splitter2_position,mixer3,mixer4,Temperatures,Pressures,enthalpies,entropies,mass_flow)
                    if (
                        hx_position.size != 0
                        and Temperatures[hx_position[0] - 1] != 0
                        and Temperatures[hx_position[1] - 1] != 0
                    ):
                        hotside_index, coldside_index = hx_side_selection(hx_position, Temperatures)
                        if (
                            Temperatures[hotside_index - 1]
                            < Temperatures[coldside_index - 1] + approach_temp
                        ):
                            print("Looping Infeasible HX1")
                            return PENALTY_VALUE
                        try:
                            (
                                Temperatures[hotside_index],
                                enthalpies[hotside_index],
                                entropies[hotside_index],
                                Temperatures[coldside_index],
                                enthalpies[coldside_index],
                                entropies[coldside_index],
                                q_hx[min(hotside_index, coldside_index)],
                            ) = HX_calculation(
                                Temperatures[hotside_index - 1],
                                Pressures[hotside_index - 1],
                                enthalpies[hotside_index - 1],
                                Temperatures[coldside_index - 1],
                                Pressures[coldside_index - 1],
                                enthalpies[coldside_index - 1],
                                approach_temp,
                                hx_pdrop,
                                mass_flow[hotside_index],
                                mass_flow[coldside_index],
                            )
                        except:
                            print("Looping HX1 calculation error")
                            return PENALTY_VALUE
                    if (
                        hx2_position.size != 0
                        and Temperatures[hx2_position[0] - 1] != 0
                        and Temperatures[hx2_position[1] - 1] != 0
                    ):
                        hotside_index2, coldside_index2 = hx_side_selection(hx2_position, Temperatures)
                        if (
                            Temperatures[hotside_index2 - 1]
                            < Temperatures[coldside_index2 - 1] + approach_temp_2
                        ):
                            print("Looping Infeasible HX2")
                            return PENALTY_VALUE
                        try:
                            (
                                Temperatures[hotside_index2],
                                enthalpies[hotside_index2],
                                entropies[hotside_index2],
                                Temperatures[coldside_index2],
                                enthalpies[coldside_index2],
                                entropies[coldside_index2],
                                q_hx[min(hotside_index2, coldside_index2)],
                            ) = HX_calculation(
                                Temperatures[hotside_index2 - 1],
                                Pressures[hotside_index2 - 1],
                                enthalpies[hotside_index2 - 1],
                                Temperatures[coldside_index2 - 1],
                                Pressures[coldside_index2 - 1],
                                enthalpies[coldside_index2 - 1],
                                approach_temp_2,
                                hx_pdrop,
                                mass_flow[hotside_index2],
                                mass_flow[coldside_index2],
                            )
                        except:
                            print("Looping HX2 calculation error")
                            return PENALTY_VALUE
        print(Temperatures,"Final Temperatures")
        print(assumed_temperature,"Final assumed temperature")
        print("Converged",Temperatures[assumed_inlet])
        
    for index in cooler_position:
        if Temperatures[index] >= Temperatures[index - 1]:
            print("Infeasible Cooler")
            return PENALTY_VALUE
    enthalpies, entropies, q_cooler = cooler_calculation(
        cooler_position,
        Temperatures,
        Pressures,
        enthalpies,
        entropies,
        q_cooler,
        cooler_pdrop,
        mass_flow,
    )

    for index in heater_position:
        if Temperatures[index] <= Temperatures[index - 1]:
            print("Infeasible Temperatures for heater")
            return PENALTY_VALUE
        else:
            enthalpies, entropies, q_heater = heater_calculation(
                heater_position,
                Temperatures,
                Pressures,
                enthalpies,
                entropies,
                q_heater,
                heater_pdrop,
                mass_flow,
            )

    total_heat = sum(q_heater)
    fg_tout = fg_calculation(fg_m, total_heat)
    cost_tur = turbine_econ(w_tur, Temperatures, cost_tur)
    cost_comp = comp_econ(w_comp, cost_comp)
    cost_cooler = cooler_econ(q_cooler, Temperatures, cost_cooler)
    if np.all(cost_cooler == 0):
        return PENALTY_VALUE
    try:
        cost_heater, fg_mlist, fg_tinlist, fg_toutlist, descending_temp = heater_econ(
            equipment,
            q_heater,
            Temperatures,
            cost_heater,
            heater_position,
            total_heat,
            fg_m,
            fg_tin,
        )
    except:
        # print("Heater calculation error")
        return PENALTY_VALUE
    if hx_position.size != 0:
        cost_hx = hx_econ(q_hx, Temperatures, cost_hx, hotside_index, coldside_index)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    if hx2_position.size != 0:
        cost_hx2 = hx_econ(q_hx, Temperatures, cost_hx, hotside_index2, coldside_index2)
        if np.all(cost_hx == 0):
            return PENALTY_VALUE
    pec = cost_tur + cost_hx + cost_cooler + cost_comp + cost_heater+cost_hx2
    prod_capacity = (sum(w_tur) - sum(w_comp)) / 1e6
    zk, cfueltot, lcoe = economics(pec, prod_capacity)
    for i, j, k, l in zip(equipment, Temperatures, mass_flow, pec):
        print(f" {i}, {j:.2f}, {k:.0f}, Cost: {l:.0f}")
    Ep = sum(w_tur) - sum(w_comp)
    c = lcoe
    thermal_efficiency = (Ep) / 40.53e6
    if thermal_efficiency < 0.1575:
        j = 100 * (0.30 - thermal_efficiency)
    else:
        j = c + 1 * max(0, 0.1 - sum(q_hx) / sum(q_heater))
    # print("Succesful Completion")
    return c

In [19]:
equipment = [1,5,2,3,5,4]
x = [78.55e5,10.7,32.25,241.30e5,10.7,411.85,93.2]
equipment = [9,6,5,1,7,4,1,5,7,6,2,3] #W2CC
x = [0.542,14.1,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,182.39711967668362,119.2]
# equipment = [9,4,7,6,7,10,4,1,5,8,5,1,8,6,2,3] #Noaman1
# x = [0.3,211.5,0.0,5.2,0.0,0.47,520.4,78e5,7.4,0.0,7.4,78e5,0.0,5.2,32.2,240e5,214.82955469678234,129.8]
equipment = [9,5,1,7,4,1,5,7,6,2,3,10,4,8,6,8] #ED4
# # x = [0.99, 23.0, 8083650.370567948, 0.0, 394.1753975282282, 8902214.157824906, 23.0, 0.0, 4.0, 32.75299068162392, 30000000.0, 0.01, 329.97434984918607, 0.0, 4.0, 0.0, 160]
# x = [0.542,14.1,78.55e5,0,483.75,79.33e5,14.1,0,14.1,32.25,240e5,0.5,400,0,14.1,0,337.987450204182,100]
x = [0.99, 4.0, 7820253.851898901, 0.0, 530.0, 24328973.579785462, 4.0, 0.0, 23.0, 32.0, 30000000.0, 0.01, 180.0, 0.0, 23.0, 0.0, 500, 160]
print(objective_function2(x,equipment))

500 initial assumption
[  0.           0.           0.           0.         530.
 505.03561971   0.           0.           0.          32.
  78.24382879  78.24382879 180.          78.24382879   0.
   0.        ] Initial Temperatures
Entering Looping
[250.61398434 252.84747107 131.63205234 250.61398434 530.
 505.03561971 254.61398434 132.45682904 101.24382879  32.
  78.24382879  78.24382879 180.          78.24382879  97.19132093
  97.93591409] Final Temperatures
500 Final assumed temperature
Converged 250.61398433587067
 9, 250.61, 158, Cost: 0
 5, 252.85, 158, Cost: 157713
 1, 131.63, 158, Cost: 3276652
 7, 250.61, 2, Cost: 0
 4, 530.00, 2, Cost: 35585
 1, 505.04, 2, Cost: 33418
 5, 254.61, 2, Cost: 0
 7, 132.46, 160, Cost: 0
 6, 101.24, 160, Cost: 1045338
 2, 32.00, 160, Cost: 2154764
 3, 78.24, 160, Cost: 2552920
 10, 78.24, 2, Cost: 0
 4, 180.00, 2, Cost: 3842
 8, 78.24, 158, Cost: 0
 6, 97.19, 158, Cost: 0
 8, 97.94, 160, Cost: 0
199.28460401225595
